In [1]:
import tensorflow as tf
import numpy as np 

In [2]:
print(tf.__version__)

2.15.0


In [3]:
#Q1) 정규 배치화를 사용한 모델을 만들어 보자.  
model = tf.keras.Sequential([
   
    tf.keras.layers.BatchNormalization( name ='bn01' , input_shape=(1,))
])
model.summary() 
print (model.layers[0].trainable)

#[확인 ]   trainable = True (훈련모드)  미니 배치의 평균 및 분산으로 정규화 하겠다.  gamma,beta가 업데이트된다
# moving_mean ,# moving_variance -> 평균 및 분산의 이동 평균이 업데이트 된다.

# trainable = False (추론 = 예측,분석모드)   
# moving_mean ,# moving_variance -> 
#평균 및 분산의 이동 평균이 업데이트 되지 않은 상태에서  moving_mean,  moving_varianc로 정규화 하겠다.  


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bn01 (BatchNormalization)   (None, 1)                 4         
                                                                 
Total params: 4 (16.00 Byte)
Trainable params: 2 (8.00 Byte)
Non-trainable params: 2 (8.00 Byte)
_________________________________________________________________
True


In [4]:
#Q2) 모델이 가진 속성을 이용해서 가중치, 바이어스 등의 값을  리턴 받아 보자.  
for w in model.trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))
    
for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))

bn01/gamma:0                  [1.]
bn01/beta:0                   [0.]
bn01/moving_mean:0            [0.]
bn01/moving_variance:0        [1.]


In [5]:
model.layers[0].trainable = False 

print(model.layers[0].trainable_weights )

for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))

[]
bn01/gamma:0                  [1.]
bn01/beta:0                   [0.]
bn01/moving_mean:0            [0.]
bn01/moving_variance:0        [1.]


In [6]:
#Q3) 실제 동작을 확인 해보자.     moving_mean_initializer='zeros',   moving_variance_initializer='ones',
# training=True 일때 확인  
model = tf.keras.Sequential([   
    tf.keras.layers.BatchNormalization( name ='bn01' , input_shape=(1,))
])

for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))

#3-1, 100개의 요소가 1개만 있는 데이터를 사용해보자.  (1,1)차원의 2차원으로  확인 
a = np.array([[100]]).astype('float32')
print(a , a.shape)

#3-2  학습 모델로 호출하게 되면 출력은 초기의 0, 업데이트 되는 것을 확인 할  수 있다.  
print(model(a, training=True))

for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))

#3-3  데이터를 계속 호출해서 데이터의 평균과 분산이  0에서 가까워 지는 것을 확인 하자.  
for i  in range(1000):
    model(a, training=True)
    
for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))
    
model.predict(a)    

bn01/moving_mean:0            [0.]
bn01/moving_variance:0        [1.]
[[100.]] (1, 1)
tf.Tensor([[0.]], shape=(1, 1), dtype=float32)
bn01/moving_mean:0            [1.]
bn01/moving_variance:0        [0.99]
bn01/moving_mean:0            [99.99573]
bn01/moving_variance:0        [4.273953e-05]
1/1 [==============================] - 0s 250ms/step


array([[0.13232422]], dtype=float32)

In [7]:
#Q4) 실제 동작을 확인 해보자.     moving_mean_initializer='zeros',   moving_variance_initializer='ones',
# training=False 일때 확인  

model = tf.keras.Sequential([   
    tf.keras.layers.BatchNormalization( name ='bn01' , input_shape=(1,))
])

for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))

#3-1, 100개의 요소가 1개만 있는 데이터를 사용해보자.  (1,1)차원의 2차원으로  확인 
a = np.array([[100]]).astype('float32')
print(a , a.shape)

#3-2  학습 모델로 호출하게 되면 출력은 초기의 0, 업데이트 되는 것을 확인 할  수 있다.  
print(model(a, training=False))

for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))

#3-3  데이터를 계속 호출해서 데이터의 평균과 분산이  0에서 가까워 지는 것을 확인 하자.  
for i  in range(1000):
    model(a, training=False)
    
for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))
    
model.predict(a)
print((100-0) / np.sqrt(1+0.001))

bn01/moving_mean:0            [0.]
bn01/moving_variance:0        [1.]
[[100.]] (1, 1)
tf.Tensor([[99.95004]], shape=(1, 1), dtype=float32)
bn01/moving_mean:0            [0.]
bn01/moving_variance:0        [1.]
bn01/moving_mean:0            [0.]
bn01/moving_variance:0        [1.]
1/1 [==============================] - 0s 56ms/step
99.95003746877732


In [11]:
#Q5)  training=False 일 경우 훈련 실행하는 fit() 은 추론 모드로 동작된다.  
# fit()을 실행할 때   bn01/moving_mean, bn01/moving_variances는 갱신되지 않는다.

model = tf.keras.Sequential([   
    tf.keras.layers.BatchNormalization( name ='bn03' , input_shape=(1,))
])

model.layers[0].trainable =False #학습중에 가중치 업데이트 하지 않겠다 -> 가중치 고정

model.compile(optimizer ='adam' , loss = 'mean_squared_error')
model.fit(a,a)

for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))


1/1 [==============================] - 0s 159ms/step - loss: 0.0025
bn03/gamma:0                  [1.]
bn03/beta:0                   [0.]
bn03/moving_mean:0            [0.]
bn03/moving_variance:0        [1.]


In [14]:
#Q6)trainable =True
model = tf.keras.Sequential([   
    tf.keras.layers.BatchNormalization( name ='bn04' , input_shape=(1,))
])

model.layers[0].trainable =True

model.compile(optimizer ='adam' , loss = 'mean_squared_error')
model.fit(a,a)


for w in model.non_trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))

    
for w in model.trainable_weights:
    print('{:<30}{}'. format (w.name , w.numpy()))
    
    
### 추론에서는 모든 모델의 모든 가중치가 고정되어야 한다
### 전이 학습 시나리오에서는 BatchNormalization 레이어를 고정시켜 기존의 통계 (평균 및 분산)을 유지하면서 
### 다른 레이어를 학습 시켜야 한다

1/1 [==============================] - 1s 752ms/step - loss: 10000.0000
bn04/moving_mean:0            [1.]
bn04/moving_variance:0        [0.99]
bn04/gamma:0                  [1.]
bn04/beta:0                   [0.00099999]
